In [1]:
import numpy as np 
import pandas as pd 
import datetime
from datetime import datetime, date
from tqdm import tqdm_notebook as tqdm
import time

from multiprocessing import Pool
from functools import partial
import VWAP_feature_function as func

In [2]:
data = pd.read_csv('D:\\庫存健診開發\\data\\processed\\TWSE_stock_processed_unfixed.csv', converters={'ts': str, 'StockName': str, 'StockNo': str})
data['ts'] = pd.to_datetime(data['ts'])
data

,ts,StockNo,StockName,open,high,low,close,return,volatility,vol,...,industry_close,industry_return,industry_vol,foreign_buy,investment_buy,dealer_buy,foreign_ratio,investment_ratio,dealer_ratio,corporation_ratio
0,2007-07-02,1101,台泥,38.2,38.20,37.00,37.50,-1.57,3.15,10222.0,...,124.56,0.02,44467.0,-3917.0,-765.0,158.0,47.42,0.78,0.28,48.48
1,2007-07-03,1101,台泥,37.7,37.80,37.05,37.40,-0.27,2.00,9831.0,...,124.54,-0.02,76827.0,-1035.0,0.0,1255.0,47.39,0.78,0.32,48.49
2,2007-07-04,1101,台泥,37.5,38.40,37.35,37.70,0.80,2.81,11124.0,...,125.13,0.47,48887.0,1057.0,680.0,198.0,47.43,0.80,0.32,48.55
3,2007-07-05,1101,台泥,37.7,38.00,37.00,37.00,-1.86,2.65,15043.0,...,123.23,-1.52,55899.0,-8619.0,601.0,74.0,47.16,0.82,0.33,48.31
4,2007-07-06,1101,台泥,37.0,38.65,36.75,38.55,4.19,5.14,25513.0,...,125.67,1.98,103863.0,5288.0,1485.0,1155.0,47.32,0.87,0.36,48.55
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2369242,2019-09-17,9958,世紀鋼,78.9,78.90,77.10,77.30,-0.26,2.32,3050.0,...,101.55,-1.14,31349.0,508.0,0.0,-358.0,2.22,1.30,1.45,4.97
2369243,2019-09-18,9958,世紀鋼,77.9,79.30,77.50,77.70,0.91,2.34,5940.0,...,101.80,0.25,33530.0,-291.0,0.0,144.0,2.06,1.30,1.52,4.88
2369244,2019-09-19,9958,世紀鋼,78.4,78.40,76.20,76.30,-1.80,2.83,4011.0,...,101.59,-0.21,22747.0,64.0,0.0,-161.0,2.08,1.30,1.44,4.82
2369245,2019-09-20,9958,世紀鋼,76.6,77.00,75.80,75.80,-0.66,1.57,1742.0,...,102.10,0.50,55930.0,144.0,0.0,21.0,2.60,1.30,1.45,5.35


In [3]:
stock_list = data['StockNo'].unique().tolist()
d1 = data[data.StockNo.isin(stock_list[:180])]
d2 = data[data.StockNo.isin(stock_list[180:360])]
d3 = data[data.StockNo.isin(stock_list[360:540])]
d4 = data[data.StockNo.isin(stock_list[540:720])]
d5 = data[data.StockNo.isin(stock_list[720:])]

In [6]:
for i, d in enumerate([d1, d2, d3, d4, d5]):
    d.to_csv(f'D:\\庫存健診開發\\feature_{i}.csv', index=False)

In [3]:
columns_dict = {'lag': ['index_close', 'industry_close', 'close', 'VWAP', 'VWAP_day5'],
               'return': ['index', 'industry', 'close', 'VWAP', 'VWAP_day5'],
               'ratio': ['index_close', 'index_vol', 'index_return', 'industry_close', 'industry_vol', 'industry_return', 'VWAP_return', 'close_return', 'vol', 'VWAP', 'VWAP_day5', 'VWAP_day5_return'],
               'momentum': ['index_close', 'index_vol', 'index_return', 'industry_close', 'industry_vol', 'industry_return', 'VWAP_return', 'close_return', 'vol', 'VWAP', 'VWAP_day5', 'VWAP_day5_return'],
                'moment': ['index_close', 'index_vol', 'industry_close', 'industry_vol', 'vol', 'VWAP', 'VWAP_day5', 'VWAP_day5_return'],
                'buyer': ['foreign_buy', 'investment_buy', 'dealer_buy']
               }

In [ ]:
df_list = [group[1] for group in data.groupby(data['StockNo'])]
output_list = []

if __name__ == '__main__':
    with Pool(processes=12) as pool:
        for i, x in enumerate(tqdm(pool.imap_unordered(partial(func.separate_engineering, columns_dict=columns_dict), df_list), total=len(df_list)), 1):
            output_list.append(x)
                
df = pd.concat(output_list, axis=0)
df

In [7]:
d.to_csv('D:\\庫存健診開發\\data\\processed\\TWSE_feature_all_new.csv', index=False)

In [2]:
import os
path = 'C:\\Feature\\'
file_list = os.listdir(path)
df_list = []
for f in file_list:
    filename = path + f
    d = pd.read_csv(filename, converters={'ts': str, 'StockName': str, 'StockNo': str})
    df_list.append(d)
    
df = pd.concat(df_list, axis=0)
df

,ts,StockNo,StockName,open,high,low,close,return,volatility,vol,...,industry_pvt,foreign_pvt,investment_pvt,dealer_pvt,pvt,RSI_15,SOk5_1,SOd3,SOk15_5,SOd5
0,2007-08-14,1104,環泥,17.0,17.1,16.55,17.0,2.41,3.31,3312.0,...,2414.822719,20.276376,40.113317,145.677094,-526.121026,0.424197,0.693237,0.663900,0.382322,0.316308
1,2007-08-15,1104,環泥,16.8,16.8,16.15,16.3,-4.12,3.82,3387.0,...,-419.033581,27.887669,26.061763,134.297358,-1533.226609,0.363161,0.207221,0.435560,0.372895,0.335170
2,2007-08-16,1104,環泥,15.8,16.0,15.20,15.2,-6.75,4.91,4684.0,...,-4632.919957,84.136556,26.061763,128.026927,-1682.138578,0.261163,0.063002,0.249281,0.388269,0.352870
3,2007-08-17,1104,環泥,15.0,15.2,14.15,14.2,-6.58,6.91,4261.0,...,-6153.838613,86.084850,22.997972,118.324063,-1663.512877,0.192773,0.083706,0.166494,0.440309,0.382016
4,2007-08-20,1104,環泥,15.1,15.1,14.70,15.0,5.63,2.82,2223.0,...,-4461.700478,82.989338,7.971338,116.896775,-1863.669743,0.192773,0.270447,0.218470,0.445407,0.403146
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
459817,2019-09-09,9958,世紀鋼,76.4,76.9,75.10,75.2,-1.05,2.37,2587.0,...,-725.705239,20.186656,30.535589,38.064468,331.738249,0.691654,0.807395,0.822845,0.778816,0.668481
459818,2019-09-10,9958,世紀鋼,75.7,76.2,74.10,74.8,-0.53,2.79,2981.0,...,-414.454949,16.244066,30.535589,38.027537,311.480669,0.598361,0.365302,0.594074,0.806446,0.714469
459819,2019-09-11,9958,世紀鋼,75.6,77.2,75.00,76.6,2.41,2.94,4732.0,...,-5.836016,21.734884,30.535589,44.403342,404.716865,0.652006,0.737532,0.665803,0.825123,0.751354
459820,2019-09-12,9958,世紀鋼,77.0,77.8,76.60,76.8,0.26,1.57,3440.0,...,259.180753,33.513643,30.535589,44.014241,483.509127,0.681203,0.840273,0.753038,0.820258,0.774322


In [8]:
def label(row):

    if row['VWAP_after'] - row['VWAP_day5'] > 0:
        return 1
    elif row['VWAP_after'] - row['VWAP_day5'] <= 0:
        return 0
    
df_list = [group[1] for group in df.groupby(df['StockNo'])]
output_list = []

for i, d in enumerate(tqdm(df_list)):
    d = d.sort_values(by='ts')
#     d['VWAP_after'] = d['VWAP_day5'].shift(-5)
#     d['Y'] = d.apply(label, axis=1)
    d['VWAP_lag'] = d['VWAP_day5'].shift(5)
    d['VWAP_day5_return_5'] = (d['VWAP_day5'] - d['VWAP_lag']) / d['VWAP_lag']
    d = d.drop(columns=['VWAP_lag'])
    output_list.append(d)
    
df = pd.concat(output_list, axis=0)
df

,ts,StockNo,StockName,open,high,low,close,return,volatility,vol,...,foreign_pvt,investment_pvt,dealer_pvt,pvt,RSI_15,SOk5_1,SOd3,SOk15_5,SOd5,VWAP_day5_return_5
11956,2007-08-14,1101,台泥,42.0,44.0,41.20,42.80,2.39,6.70,28881.0,...,3403.523526,1269.505097,-175.248480,4646.811429,0.611272,0.758871,0.725291,0.721390,0.647950,NaN
11957,2007-08-15,1101,台泥,41.5,42.6,39.85,39.85,-6.89,6.43,18071.0,...,3707.555034,1234.783707,-189.224958,2624.236833,0.538065,0.322895,0.524093,0.691707,0.662535,NaN
11958,2007-08-16,1101,台泥,38.5,38.5,37.10,37.10,-6.90,3.51,30715.0,...,4856.448178,1254.673033,-188.201218,271.929982,0.420754,0.034938,0.279515,0.581916,0.635662,NaN
11959,2007-08-17,1101,台泥,36.1,37.0,34.85,36.00,-2.96,5.80,36804.0,...,5282.798946,1258.026027,-163.373507,-966.451392,0.349502,0.093699,0.186607,0.473700,0.581675,NaN
11960,2007-08-20,1101,台泥,38.4,38.4,37.20,37.80,5.00,3.33,24456.0,...,4523.052314,1158.159980,-148.807704,47.910645,0.419417,0.318759,0.252683,0.400546,0.521299,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
459817,2019-09-09,9958,世紀鋼,76.4,76.9,75.10,75.20,-1.05,2.37,2587.0,...,20.186656,30.535589,38.064468,331.738249,0.691654,0.807395,0.822845,0.778816,0.668481,0.075285
459818,2019-09-10,9958,世紀鋼,75.7,76.2,74.10,74.80,-0.53,2.79,2981.0,...,16.244066,30.535589,38.027537,311.480669,0.598361,0.365302,0.594074,0.806446,0.714469,0.048711
459819,2019-09-11,9958,世紀鋼,75.6,77.2,75.00,76.60,2.41,2.94,4732.0,...,21.734884,30.535589,44.403342,404.716865,0.652006,0.737532,0.665803,0.825123,0.751354,0.035145
459820,2019-09-12,9958,世紀鋼,77.0,77.8,76.60,76.80,0.26,1.57,3440.0,...,33.513643,30.535589,44.014241,483.509127,0.681203,0.840273,0.753038,0.820258,0.774322,0.030735


In [10]:
df['VWAP_day5_return_5'] = round(df['VWAP_day5_return_5'] * 100, 2)

In [11]:
df[['ts', 'StockNo', 'VWAP_day5', 'VWAP_day5_return', 'VWAP_day5_return_5', 'Y']].iloc[-50:]

,ts,StockNo,VWAP_day5,VWAP_day5_return,VWAP_day5_return_5,Y
459772,2019-07-05,9958,67.859629,1.06,2.90,1.0
459773,2019-07-08,9958,68.235612,0.55,3.55,1.0
459774,2019-07-09,9958,69.450160,1.78,5.35,1.0
459775,2019-07-10,9958,69.779639,0.47,5.59,1.0
459776,2019-07-11,9958,70.157974,0.54,4.48,1.0
459777,2019-07-12,9958,70.334217,0.25,3.65,1.0
459778,2019-07-15,9958,70.363957,0.04,3.12,1.0
459779,2019-07-16,9958,71.825265,2.08,3.42,1.0
459780,2019-07-17,9958,75.776406,5.50,8.59,1.0
459781,2019-07-18,9958,76.810125,1.36,9.48,1.0


In [14]:
d = df[df.Y.notnull()]
d = d[d['VWAP_day5_return_5'].notnull()]

In [17]:
d.to_csv('D:\\庫存健診開發\\data\\processed\\TWSE_feature_all_5day.csv', index=False)

In [15]:
d.isna().values.any()

False

In [16]:
d

,ts,StockNo,StockName,open,high,low,close,return,volatility,vol,...,foreign_pvt,investment_pvt,dealer_pvt,pvt,RSI_15,SOk5_1,SOd3,SOk15_5,SOd5,VWAP_day5_return_5
11961,2007-08-21,1101,台泥,37.50,39.90,36.80,37.5,-0.79,8.20,42969.0,...,4118.371040,1138.194426,-106.652476,176.739261,0.486939,0.459669,0.356176,0.310119,0.450905,-9.31
11962,2007-08-22,1101,台泥,37.35,38.40,36.80,38.2,1.87,4.27,23201.0,...,4014.678434,1136.459691,-104.736206,-245.989525,0.486939,0.591611,0.473894,0.277664,0.393158,-9.45
11963,2007-08-23,1101,台泥,39.60,40.00,39.25,40.0,4.71,1.96,30456.0,...,4403.772485,1231.769792,-86.190952,1427.473280,0.558479,0.953949,0.713921,0.328728,0.371681,-6.01
11964,2007-08-24,1101,台泥,40.00,40.85,39.65,39.8,-0.50,3.00,32479.0,...,3990.608891,1374.195281,-90.830337,2613.957456,0.592940,0.831259,0.772590,0.438425,0.393929,-0.82
11965,2007-08-27,1101,台泥,41.00,41.00,39.20,39.5,-0.75,4.52,16646.0,...,3705.511740,1349.949967,-160.328628,4073.934540,0.566202,0.719624,0.746107,0.470497,0.419452,1.66
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
459812,2019-09-02,9958,世紀鋼,69.40,71.60,68.70,71.6,3.32,4.18,1980.0,...,10.399445,30.535589,5.001610,-135.261368,0.433530,0.618687,0.459431,0.251528,0.272107,-2.98
459813,2019-09-03,9958,世紀鋼,71.70,75.80,71.30,74.1,3.49,6.28,7286.0,...,7.350933,30.535589,35.547803,270.566486,0.669457,0.824588,0.642009,0.472498,0.338904,0.48
459814,2019-09-04,9958,世紀鋼,75.00,75.40,74.40,74.9,1.08,1.35,4513.0,...,2.819805,30.535589,35.061530,348.691073,0.669457,0.884340,0.763175,0.644186,0.440665,3.30
459815,2019-09-05,9958,世紀鋼,75.50,75.70,74.00,74.2,-0.93,2.27,2186.0,...,14.334574,30.535589,34.972803,301.521091,0.636482,0.815226,0.789200,0.739897,0.540409,5.25
